In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../Hierarchical-Localization')

import math
from pathlib import Path
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval
from hloc.utils import read_write_model
from hloc.utils.io import read_image

import pycolmap
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pickle
from PIL import Image
from scipy.stats import norm

## Setup

In [2]:
dataset = 'South-Building'
images_path = Path(f'datasets/{dataset}/images/')
masks_path = Path(f'datasets/{dataset}/masks/')
#ground_truth_path = f'datasets/{dataset}/ground_truth.ply'

outputs_sfm = Path(f'outputs/{dataset}/sfm/')
outputs_seg = Path(f'outputs/{dataset}/segmentation/')
sfm_pairs = outputs_sfm / 'pairs-netvlad.txt'
sfm_dir = outputs_sfm / 'sfm_superpoint+superglue'

retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Run SfM
[Skip](#loading_images_and_points) this step if you already have the points3D.bin and images.txt

### Find image pairs via image retrieval

In [ ]:
retrieval_path = extract_features.main(retrieval_conf, images_path, outputs_sfm)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

### Extract and match local features

In [ ]:
feature_path = extract_features.main(feature_conf, images_path, outputs_sfm)
match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs_sfm)

### 3D reconstruction

In [ ]:
model = reconstruction.main(sfm_dir, images_path, sfm_pairs, feature_path, match_path)
# convert colmap images to hloc images (they are slightly different, this is most likely a bug, so here is the workaround)
ims={}
for image_id in model.images:
    im = model.images[image_id]
    ims[image_id] = read_write_model.BaseImage(im.image_id, 
                                               im.qvec, 
                                               im.tvec, 
                                               im.camera_id, 
                                               im.name, 
                                               [point2D.xy for point2D in im.points2D], 
                                               [point2D.point3D_id for point2D in im.points2D])

read_write_model.write_images_text(ims, sfm_dir/"images.txt")

## Loading the images and points
<a id='loading_images_and_points'></a>

In [3]:
images = read_write_model.read_images_text(sfm_dir/"images.txt")
points3D = read_write_model.read_points3D_binary(sfm_dir/"points3D.bin")
#read_write_model.write_points3D_text(points3D, 'datasets/South-Building/p3D.txt')

## Pointcloud Segmentation


### scaling


In [4]:
im = Image.open(images_path/images[1].name)
image_resolution = im.size

pkl_resolution = (512, 384)
def get_pkl_coordinates(image_coordinates):
    x = int(round((image_coordinates[0]/image_resolution[0])*pkl_resolution[0]))
    y = int(round((image_coordinates[1]/image_resolution[1])*pkl_resolution[1]))
    return (x, y)

### get 3D points and corresponding lables with certainty


In [5]:
point3D_masks = {} # dict of all points3D and their corresponding mask colors from all images


def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

def normalize(x):
    s = sum(x)
    return [i/s for i in x]
print("getting points and corresponding mask colors...")
for image_id in images:
    image = images[image_id]
    data = None
    with open(masks_path/Path(image.name.replace(".JPG", ".pkl")), 'rb') as f:
        data = pickle.load(f)[0]

    no_invalid_points = 0
    no_valid_points = 0
    # iterate over points2D in image
    for i, point2D in enumerate(image.xys):
        # get certainties
        pkl_x, pkl_y = get_pkl_coordinates((int(point2D[1]), int(point2D[0])))
        certainties_raw = data[pkl_x][pkl_y]
        certainties_raw[3] = min(certainties_raw) # make sky lowest certainty
        certainties = softmax(certainties_raw)
        # add certainties to corresponding point3D
        point3D_id = image.point3D_ids[i]
        # points with this ID are invalid
        if point3D_id == float("1.8446744073709552e+19"):
            no_invalid_points += 1
            continue
        else:
            no_valid_points += 1
        if not point3D_id in point3D_masks:
            point3D_masks[point3D_id] = certainties
        else:
            point3D_masks[point3D_id] = [sum(x) for x in zip(point3D_masks[point3D_id], certainties)]
    #print(f"invalid: {no_invalid_points} valid: {no_valid_points}")

# normalize
print("normalizing...")
for point3D_id in point3D_masks:
    point3D_masks[point3D_id]=normalize(point3D_masks[point3D_id])

print("done")

getting points and corresponding mask colors...
normalizing...
done


### knn

In [6]:
def knn(k, weight=None, absolute=False):
    print(f"running knn with k={k}")
    # the neighbor furthest away will be weighted as 0 so we have to increase k by 1
    if weight=="linear":
        k+=1
    mask_colors = [[0, 0, 0], [120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50], [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255], [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7], [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82], [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3], [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255], [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220], [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224], [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255], [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7], [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153], [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255], [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0], [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255], [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255], [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255], [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0], [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0], [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255], [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255], [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20], [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255], [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255], [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255], [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0], [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0], [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255], [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112], [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160], [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163], [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0], [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0], [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255], [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204], [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255], [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255], [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194], [102, 255, 0], [92, 0, 255]]
    points3D_segmented = {} # dict of points3D (segmentational rgb applied)
    index_dict = {}
    id_dict = {}
    

    # knn on 3D point cloud
    number_of_points = len(points3D.items())
    
    coordinates = np.zeros(shape=(number_of_points,3))
    # putting all 3D coordinates into the array
    for i, point3D_id in enumerate(points3D):
        coordinates[i] = points3D[point3D_id].xyz
        # making sure we can convert between point IDs and indices in the array later on
        index_dict[point3D_id] = i
        id_dict[i] = point3D_id

    # running knn
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(coordinates)
    distances, indices = nbrs.kneighbors(coordinates)
    
    # creating segmented pointcloud
    print("creating segmented pointcloud...")
    for point3D_id in point3D_masks:
        
        index_in_array = index_dict[point3D_id]
        neighbor3D_idxs = indices[index_in_array]
        neighbor3D_distances = distances[index_in_array]

        #print(np.argmax(point3D_masks[point3D_id]))

        certainties = [0]*len(mask_colors)
        if weight=="linear" or weight=="normal":
            dist_max = np.amax(neighbor3D_distances)
        
        # iterate over all k neighbors of point (including itself) and add certainties
        for neighbor3D_idx, neighbor3D_dist in zip(neighbor3D_idxs, neighbor3D_distances):
            certainties_neighbor = point3D_masks[id_dict[neighbor3D_idx]]
            if not absolute and weight=="linear":
                factor = (dist_max-neighbor3D_dist)/dist_max
                certainties_neighbor = (np.array(certainties_neighbor)*factor).tolist()
            #elif weight=="normal":
            if absolute:
                dist_max = 0.5
                factor = 1
                #linear
                if weight=="linear":
                    factor = (dist_max-neighbor3D_dist)/dist_max
                    factor = factor * (factor>0)
                #print(factor)
                #normal
                elif weight=="normal":
                    factor = norm.pdf(neighbor3D_dist,0,dist_max*0.4)*dist_max
                #1/(1+d²)
                elif weight=="d_square":
                    second_factor = 5
                    factor = 1/(1+(neighbor3D_dist*second_factor)**2)
                #print(factor)
                certainties_neighbor = (np.array(certainties_neighbor)*factor).tolist()
                
            
            certainties = [sum(x) for x in zip(certainties_neighbor, certainties)]

        
        # determine color with highest certainty
        mask_color = np.argmax(certainties)
        rgb = mask_colors[mask_color]
        #create point3D
        point3D = points3D[point3D_id]
        # cutting the pointcloud for fun
        #if point3D.xyz[2] > 0:
        #    continue
        points3D_segmented[point3D_id] = read_write_model.Point3D(id=point3D_id, 
                                                                  xyz=point3D.xyz, 
                                                                  rgb=rgb, 
                                                                  error=point3D.error, 
                                                                  image_ids=[], 
                                                                  point2D_idxs=[])

    # write file
    file_name = f"{dataset}_k{k}"
    if weight:
        file_name += "_euc"
    
    # create path if not exists
    Path(outputs_seg).mkdir(parents=True, exist_ok=True)
    read_write_model.write_points3D_text(points3D_segmented, outputs_seg/Path(file_name + ".txt"))
    print(f"file '{file_name}' written")
    
    # adjust output (so we can open it in meshlab)
    
    #file_name = "South-Building_k5"
    count = 0
    with open(outputs_seg/Path(file_name + ".txt")) as fp:
        Lines = fp.readlines()
        for line in Lines:
            Lines[count] = Lines[count].split(" ", 1)[1].rsplit(' ', 2)[0] + "\n"
            if count < 3:
                Lines[count] = "#\n"
            count += 1

        with open(outputs_seg/Path(file_name + ".txt"), 'w') as fp:
            fp.writelines(Lines)
    
    return points3D_segmented
    

## Evaluation

In [7]:
'''
# adding stuff to the ground truth file so we can use read_write_model.read_points3D_text to open it
count = 0
with open('datasets/South-Building/ground_truth.txt') as fp:
    Lines = fp.readlines()
    for line in Lines:
        Lines[count] = f"{count} {Lines[count][:-1]} 0.0\n"
        count += 1

    with open('datasets/South-Building/ground_truth_adjusted.txt', 'w') as fp:
        fp.writelines(Lines)
'''

'\n# adding stuff to the ground truth file so we can use read_write_model.read_points3D_text to open it\ncount = 0\nwith open(\'datasets/South-Building/ground_truth.txt\') as fp:\n    Lines = fp.readlines()\n    for line in Lines:\n        Lines[count] = f"{count} {Lines[count][:-1]} 0.0\n"\n        count += 1\n\n    with open(\'datasets/South-Building/ground_truth_adjusted.txt\', \'w\') as fp:\n        fp.writelines(Lines)\n'

### Load Ground Truth

In [8]:
ground_truth = read_write_model.read_points3D_text('datasets/South-Building/ground_truth_adjusted.txt')

Get dictionary containing point3D IDs as keys and the corresponding colors in the ground truth as values

In [9]:
def get_color_dict(points_1, points_2):
    # returns a dict where the keys are the point IDs in points_1 
    # and the values are the colors of the respective points in points_2
    index_list = {}
    id_list = {}
    color_dict = {}
    number_of_points_1 = len(points_1.items())
    number_of_points_2 = len(points_2.items())
    
    if number_of_points_1 != number_of_points_2:
        return
    
    coordinates = np.zeros(shape=(number_of_points_1*2,3))
    for i, id in enumerate(points_1):
        coordinates[i] = points_1[id].xyz
        index_list[id] = i
        id_list[i] = id
        
    for i, id in enumerate(points_2):
        coordinates[i+number_of_points_1] = points_2[id].xyz
        index_list[id] = i+number_of_points_1
        id_list[i+number_of_points_1] = id
    
    #print(coordinates)

    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(coordinates)
    indices = nbrs.kneighbors(coordinates)[1]
    # we only need the first half
    indices = np.split(indices, 2)[0]
    
    for j in indices:
        if not ((j[0]<37173 and j[1]>=37173) or (j[1]<37173 and j[0]>=37173)):
            print("error")
        # j[0] is index of point from points_1 cloud in array
        # id the point had in points_1:
        point_1_id = id_list[j[0]]
        # id the point had in points_2:
        point_2_id = id_list[j[1]]
        if j[0]<j[1]:
            # add to color list:
            color_dict[point_1_id] = points_2[point_2_id].rgb
        else:
            # add to color list:
            color_dict[point_2_id] = points_2[point_1_id].rgb
        #print(id_list[j[0]])
        #print(j)
    
    return color_dict

# this is what we need!
color_dict = get_color_dict(points3D, ground_truth)

You can [skip](#check_accuracy) this step. It is optional to test if everything went well

In [ ]:
# using the color list to create a pointcloud
# if everthing went well, this pointcloud will be identical with the ground truth we loaded
new_ground_truth = {}
for point3D_id in points3D:
    point = points3D[point3D_id]
    new_ground_truth[point3D_id] = read_write_model.Point3D(id=point3D_id, 
                                                                  xyz=point.xyz, 
                                                                  rgb=color_dict[point3D_id], 
                                                                  error=point.error, 
                                                                  image_ids=[], 
                                                                  point2D_idxs=[])

read_write_model.write_points3D_text(new_ground_truth, "datasets/South-Building/new_ground_truth.txt")
# adjust output (so we can open it in meshlab)
    
#file_name = "South-Building_k5"
count = 0
with open("datasets/South-Building/new_ground_truth.txt") as fp:
    Lines = fp.readlines()
    for line in Lines:
        Lines[count] = Lines[count].split(" ", 1)[1].rsplit(' ', 2)[0] + "\n"
        if count < 3:
            Lines[count] = "#\n"
        count += 1
    file = open("datasets/South-Building/new_ground_truth.txt",'w')
    file.writelines(Lines)

### Check accuracy of pointcloud
<a id='check_accuracy'></a>

In [10]:
labels = {'[4, 200, 3]': 'tree',
         '[180, 120, 120]': 'building',
         '[255, 5, 153]': 'signboard',
         '[255, 61, 6]': 'railing',
         '[173, 0, 255]': 'ashbin',
         '[235, 255, 7]': 'sidewalk',
         '[0, 71, 255]': 'streetlight',
         '[31, 0, 255]': 'stairway',
         '[150, 5, 61]': 'person'}
colors = {'tree': [4, 200, 3],
         'building': [180, 120, 120],
         'signboard': [255, 5, 153],
         'railing': [255, 61, 6],
         'ashbin': [173, 0, 255],
         'sidewalk': [235, 255, 7],
         'streetlight': [0, 71, 255],
         'stairway': [31, 0, 255],
         'person': [150, 5, 61]}

# labels we consider identical
pairs = [([4, 200, 3], [204, 255, 4]), # tree and plant
        ([31, 0, 255], [255, 224, 0]), # stairway and stairs
        ([255, 184, 6], [255, 61, 6])] # fence and railing
def considered_identical(pair):
    return pair in pairs or (pair[1], pair[0]) in pairs

def check_against_ground_truth(points):
    correct_classification = 0
    incorrect_classification = 0

    for point_id in points:
        point = points[point_id]
        pt_col = point.rgb
        gt_col = list(color_dict[point_id])
        if pt_col==gt_col or considered_identical((pt_col, gt_col)):
            correct_classification += 1
    
    return correct_classification/len(points)

def check_against_ground_truth_per_label(points):
    from collections import namedtuple
    label_dict = {} # pairs of number of correct and incorrect labeling per label
    label_accuracy = {} # accuracy per label
    for point_id in points:
        point = points[point_id]
        pt_col = point.rgb
        gt_col = list(color_dict[point_id])
        gt_col_str = str(gt_col)
        if not gt_col_str in label_dict:
            label_dict[gt_col_str] = (0, 0)
        if pt_col==gt_col or considered_identical((pt_col, gt_col)):
            label_dict[gt_col_str] = (label_dict[gt_col_str][0]+1, label_dict[gt_col_str][1])
        else:
            label_dict[gt_col_str] = (label_dict[gt_col_str][0], label_dict[gt_col_str][1]+1)
    for color in label_dict:
        correct, incorrect = label_dict[color]
        label_accuracy[labels[color]] = correct / (correct + incorrect)
    return label_accuracy

Here we run the pointcloud segmentation for the first time without knn and check its accuracy.

In [18]:
points3D_segmented = knn(1)

running knn with k=1
creating segmented pointcloud...
file 'South-Building_k1' written


In [21]:
print(f"Accuracy: {int(round(check_against_ground_truth(points3D_segmented), 2)*100)}%")

Accuracy: 90%


We can also check the accuracy of each label.

In [22]:
print(check_against_ground_truth_per_label(points3D_segmented))

{'tree': 0.8072696534234995, 'building': 0.9729930721358957, 'signboard': 0.19594594594594594, 'railing': 0.12101910828025478, 'ashbin': 0.0, 'sidewalk': 0.6031331592689295, 'streetlight': 0.5063829787234042, 'stairway': 0.6606786427145709, 'person': 0.7916666666666666}


### Run evaluation on all relevant k values

In [23]:
def run_evaluation(weight):
    ks = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    #ks = [1, 2, 4, 8]
    accuracies = []
    for k in ks:
        print("__________________________")
        points3D_segmented = knn(k, weight=weight, absolute=True)
        accuracy = check_against_ground_truth(points3D_segmented)
        accuracies.append(accuracy)
        #print(f"accuracy: {round(accuracy, 3)*100}%")
    return accuracies



Run the evaluation on 9 different k-counts and get the accuracies for each (this should take about 10min)

In [24]:
acc = run_evaluation(None)

__________________________
running knn with k=1
creating segmented pointcloud...
file 'South-Building_k1' written
__________________________
running knn with k=2
creating segmented pointcloud...
file 'South-Building_k2' written
__________________________
running knn with k=4
creating segmented pointcloud...
file 'South-Building_k4' written
__________________________
running knn with k=8
creating segmented pointcloud...
file 'South-Building_k8' written
__________________________
running knn with k=16
creating segmented pointcloud...
file 'South-Building_k16' written
__________________________
running knn with k=32
creating segmented pointcloud...
file 'South-Building_k32' written
__________________________
running knn with k=64
creating segmented pointcloud...
file 'South-Building_k64' written
__________________________
running knn with k=128
creating segmented pointcloud...
file 'South-Building_k128' written
__________________________
running knn with k=256
creating segmented pointclou

Don't run this. It takes too long

In [88]:
weights = "linear", "normal", "d_square"

for weight in weights:
    acc = run_evaluation(weight)
    print(acc)

__________________________
running knn with k=1
creating segmented pointcloud...
file 'South-Building_k2_euc' written
__________________________
running knn with k=2
creating segmented pointcloud...
file 'South-Building_k3_euc' written
__________________________
running knn with k=4
creating segmented pointcloud...
file 'South-Building_k5_euc' written
__________________________
running knn with k=8
creating segmented pointcloud...
file 'South-Building_k9_euc' written
__________________________
running knn with k=16
creating segmented pointcloud...
file 'South-Building_k17_euc' written
__________________________
running knn with k=32
creating segmented pointcloud...
file 'South-Building_k33_euc' written
__________________________
running knn with k=64
creating segmented pointcloud...
file 'South-Building_k65_euc' written
__________________________
running knn with k=128
creating segmented pointcloud...
file 'South-Building_k129_euc' written
__________________________
running knn with k=

### Tests I ran in the past

Ignore this

In [101]:
'''
# without normalization
accuracies_no_weight_no_normalization = [0.8277244236408146, 0.8374626745218303, 0.843085034837113, 0.8450488257606327, 0.8501600624108896, 0.8554327065343126, 0.8577193124041643, 0.8516665321604391, 0.8417937750517849]
accuracies_linear_weitht_no_normalization = [0.8277244236408146, 0.8327011540634331, 0.8387539343071584, 0.8421703924891722, 0.8456944556532967, 0.8524466682807414, 0.8567239663196407, 0.8578538186318027, 0.8546256691684825]
# with normalization
accuracies_no_weight = [0.8277244236408146, 0.8385118230974095, 0.843085034837113, 0.8460172705996287, 0.8529846931912948, 0.857907621122858, 0.8596562020821564, 0.856212842654615, 0.8440265784305814]
accuracies_linear_weight = [0.8277244236408146, 0.8309794743496624, 0.8354181798617276, 0.8412826513867592, 0.8470664191752079, 0.8540607430124014, 0.8584994485244667, 0.8597100045732118, 0.8564818551098916]
# no sky and merging some labels
accuracies = [0.9013800338955693, 0.9104995561294488, 0.915745299007344, 0.9187582385064429, 0.9247572162591128, 0.9261291797810238, 0.9277163532671563, 0.9196997820999112, 0.901406935141097]
accuracies_weighted = [0.9013800338955693, 0.9041239609393915, 0.9091275926075377, 0.9141581255212117, 0.9198611895730773, 0.9248379199956958, 0.9270438221289645, 0.9273397358297689, 0.9206682269389073]
accuracies_labels = [{'tree': 0.8072696534234995, 'building': 0.9729930721358957, 'signboard': 0.19594594594594594, 'railing': 0.12101910828025478, 'ashbin': 0.0, 'sidewalk': 0.6031331592689295, 'streetlight': 0.5063829787234042, 'stairway': 0.6606786427145709, 'person': 0.7916666666666666}, {'tree': 0.8285080304311073, 'building': 0.9766722768014404, 'signboard': 0.24324324324324326, 'railing': 0.07006369426751592, 'ashbin': 0.0, 'sidewalk': 0.6122715404699739, 'streetlight': 0.5957446808510638, 'stairway': 0.6846307385229541, 'person': 0.7083333333333334}, {'tree': 0.8402366863905325, 'building': 0.9805080433676465, 'signboard': 0.25675675675675674, 'railing': 0.05732484076433121, 'ashbin': 0.0, 'sidewalk': 0.6005221932114883, 'streetlight': 0.5872340425531914, 'stairway': 0.6766467065868264, 'person': 0.6666666666666666}, {'tree': 0.8556635672020287, 'building': 0.9809385885944656, 'signboard': 0.27702702702702703, 'railing': 0.01910828025477707, 'ashbin': 0.0, 'sidewalk': 0.5430809399477807, 'streetlight': 0.5531914893617021, 'stairway': 0.6986027944111777, 'person': 0.5}, {'tree': 0.8794378698224852, 'building': 0.9819953814239305, 'signboard': 0.3344594594594595, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.4595300261096606, 'streetlight': 0.6212765957446809, 'stairway': 0.7065868263473054, 'person': 0.5416666666666666}, {'tree': 0.8872569737954353, 'building': 0.9836784218560414, 'signboard': 0.30405405405405406, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.3681462140992167, 'streetlight': 0.7021276595744681, 'stairway': 0.720558882235529, 'person': 0.0}, {'tree': 0.900570583262891, 'building': 0.9831304552037262, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.30678851174934724, 'streetlight': 0.7531914893617021, 'stairway': 0.6806387225548902, 'person': 0.0}, {'tree': 0.8877852916314455, 'building': 0.9848917765861678, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.16318537859007834, 'streetlight': 0.3574468085106383, 'stairway': 0.6427145708582834, 'person': 0.0}, {'tree': 0.849323753169907, 'building': 0.9847352146855063, 'signboard': 0.0, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.08224543080939947, 'streetlight': 0.0425531914893617, 'stairway': 0.4750499001996008, 'person': 0.0}]
accuracies_labels_weighted = [{'tree': 0.8072696534234995, 'building': 0.9729930721358957, 'signboard': 0.19594594594594594, 'railing': 0.12101910828025478, 'ashbin': 0.0, 'sidewalk': 0.6031331592689295, 'streetlight': 0.5063829787234042, 'stairway': 0.6606786427145709, 'person': 0.7916666666666666}, {'tree': 0.8142434488588335, 'building': 0.9737758816392031, 'signboard': 0.20945945945945946, 'railing': 0.09554140127388536, 'ashbin': 0.0, 'sidewalk': 0.6148825065274152, 'streetlight': 0.5319148936170213, 'stairway': 0.6646706586826348, 'person': 0.75}, {'tree': 0.8265004226542688, 'building': 0.9759286077732984, 'signboard': 0.22972972972972974, 'railing': 0.08917197452229299, 'ashbin': 0.0, 'sidewalk': 0.6227154046997389, 'streetlight': 0.5531914893617021, 'stairway': 0.6666666666666666, 'person': 0.6666666666666666}, {'tree': 0.8387573964497042, 'building': 0.9784727386590473, 'signboard': 0.2635135135135135, 'railing': 0.05732484076433121, 'ashbin': 0.0, 'sidewalk': 0.6122715404699739, 'streetlight': 0.5446808510638298, 'stairway': 0.6846307385229541, 'person': 0.75}, {'tree': 0.8583051563820795, 'building': 0.9808994481193002, 'signboard': 0.27702702702702703, 'railing': 0.006369426751592357, 'ashbin': 0.0, 'sidewalk': 0.5652741514360313, 'streetlight': 0.5574468085106383, 'stairway': 0.6926147704590818, 'person': 0.6666666666666666}, {'tree': 0.8776415891800508, 'building': 0.9825433480762457, 'signboard': 0.2533783783783784, 'railing': 0.01910828025477707, 'ashbin': 0.0, 'sidewalk': 0.5, 'streetlight': 0.6382978723404256, 'stairway': 0.6966067864271457, 'person': 0.4166666666666667}, {'tree': 0.8881022823330516, 'building': 0.9832870171043876, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.4386422976501306, 'streetlight': 0.6510638297872341, 'stairway': 0.7045908183632734, 'person': 0.0}, {'tree': 0.8967666948436179, 'building': 0.9838741242318682, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.3785900783289817, 'streetlight': 0.5361702127659574, 'stairway': 0.6786427145708582, 'person': 0.0}, {'tree': 0.8891589180050719, 'building': 0.9850091980116639, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.21279373368146215, 'streetlight': 0.33617021276595743, 'stairway': 0.6167664670658682, 'person': 0.0}]
# absolute distances instead of max_dist is furthest neighbor
accuracies_absolute_linear = [{'tree': 0.8279797125950972, 'building': 0.9765157149007789, 'signboard': 0.2533783783783784, 'railing': 0.09554140127388536, 'ashbin': 0.0, 'sidewalk': 0.6122715404699739, 'streetlight': 0.5617021276595745, 'stairway': 0.6826347305389222, 'person': 0.7083333333333334}, {'tree': 0.8354818258664413, 'building': 0.9783944577087166, 'signboard': 0.28040540540540543, 'railing': 0.05732484076433121, 'ashbin': 0.0, 'sidewalk': 0.6044386422976501, 'streetlight': 0.548936170212766, 'stairway': 0.6826347305389222, 'person': 0.6666666666666666}, {'tree': 0.8473161453930684, 'building': 0.9803123409918196, 'signboard': 0.28716216216216217, 'railing': 0.05732484076433121, 'ashbin': 0.0, 'sidewalk': 0.5822454308093995, 'streetlight': 0.574468085106383, 'stairway': 0.688622754491018, 'person': 0.6666666666666666}, {'tree': 0.8601014370245139, 'building': 0.981095150495127, 'signboard': 0.32094594594594594, 'railing': 0.03184713375796178, 'ashbin': 0.0, 'sidewalk': 0.5169712793733682, 'streetlight': 0.5574468085106383, 'stairway': 0.7045908183632734, 'person': 0.4583333333333333}, {'tree': 0.8777472527472527, 'building': 0.9825824885514111, 'signboard': 0.34121621621621623, 'railing': 0.006369426751592357, 'ashbin': 0.0, 'sidewalk': 0.47389033942558745, 'streetlight': 0.6127659574468085, 'stairway': 0.7125748502994012, 'person': 0.4166666666666667}, {'tree': 0.8852493660185968, 'building': 0.9844220908841833, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.4073107049608355, 'streetlight': 0.6340425531914894, 'stairway': 0.7145708582834331, 'person': 0.0}, {'tree': 0.8956043956043956, 'building': 0.9850874789619946, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.3772845953002611, 'streetlight': 0.5617021276595745, 'stairway': 0.6766467065868264, 'person': 0.0}, {'tree': 0.8924344885883347, 'building': 0.9863008336921211, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.33159268929503916, 'streetlight': 0.2, 'stairway': 0.6307385229540918, 'person': 0.0}, {'tree': 0.867603550295858, 'building': 0.9867705193941054, 'signboard': 0.12837837837837837, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.32114882506527415, 'streetlight': 0.20851063829787234, 'stairway': 0.564870259481038, 'person': 0.0}]
accuracies_absolute_linear = [0.9102036424286445, 0.9131896806822156, 0.9173593737390041, 0.9199418933096603, 0.9250531299599172, 0.9264788959728836, 0.927985365722433, 0.9241653888575041, 0.915664595270761]
accuracies_absolute_normal = [0.9013800338955693, 0.9100691362010062, 0.9160412127081484, 0.9189465472251366, 0.9247303150135852, 0.9265057972184112, 0.9279584644769053, 0.9252414386786109, 0.9180319048771958]
accuracies_absolute_d_square = [0.9013800338955693, 0.9101767411831168, 0.9159605089715653, 0.9191348559438302, 0.9249724262233341, 0.9266134022005219, 0.9283619831598203, 0.9248917224867511, 0.915180372851263]
'''

hello world


In [ ]:
'''
# giving it a structure I can plot
k_runs_per_label = {}
for k_run in accuracies_labels_weighted:
    for label, accuracy in k_run.items():
        if label in k_runs_per_label:
            k_runs_per_label[label] += [accuracy]
        else:
            k_runs_per_label[label] = [accuracy]
print(k_runs_per_label)
'''

### Graph Visualization

Run this code to visualize the the accuracy of the algorithm

In [25]:
import matplotlib
title = 'KNN Algorithm'
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FuncFormatter
plt.clf()

ind = range(9)
width = 0.9   # the width of the bars: can also be len(x) sequence

# Add `aling='center'` to center bars on ticks
#for label, accuracies in k_runs_per_label.items():
#    p1 = plt.plot(ind, accuracies, label=label, color=[i/255 for i in colors[label]])
plt.plot(ind, acc, label="Regular KNN")
#plt.plot(ind, accuracies_weighted, label="Linear Weighted KNN")
plt.legend()
plt.xlabel('k-Count')
plt.ylabel('Accuracy')
#plt.title(title)
#plt.axis([0, len(data), -1, max(data)])

ax = plt.gca()
plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()]) 


# Place tickmarks at every multiple of 1, i.e. at any integer
ax.xaxis.set_major_locator(MultipleLocator(1))
# Format the ticklabel to be 2 raised to the power of `x`
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, pos: int(2**x)))
# Make the axis labels rotated for easier reading
plt.gcf().autofmt_xdate()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

'''
# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
'''


plt.savefig(title+".svg")

/tmp/ipykernel_38456/1355215899.py:23: UserWarning: FixedFormatter should only be used together with FixedLocator
  plt.gca().set_yticklabels([f'{x:.0%}' for x in plt.gca().get_yticks()])


### Table Visualization

Plot a wonderfully colorful table

In [26]:
accuracies_labels = [{'tree': 0.8072696534234995, 'building': 0.9729930721358957, 'signboard': 0.19594594594594594, 'railing': 0.12101910828025478, 'ashbin': 0.0, 'sidewalk': 0.6031331592689295, 'streetlight': 0.5063829787234042, 'stairway': 0.6606786427145709, 'person': 0.7916666666666666}, {'tree': 0.8285080304311073, 'building': 0.9766722768014404, 'signboard': 0.24324324324324326, 'railing': 0.07006369426751592, 'ashbin': 0.0, 'sidewalk': 0.6122715404699739, 'streetlight': 0.5957446808510638, 'stairway': 0.6846307385229541, 'person': 0.7083333333333334}, {'tree': 0.8402366863905325, 'building': 0.9805080433676465, 'signboard': 0.25675675675675674, 'railing': 0.05732484076433121, 'ashbin': 0.0, 'sidewalk': 0.6005221932114883, 'streetlight': 0.5872340425531914, 'stairway': 0.6766467065868264, 'person': 0.6666666666666666}, {'tree': 0.8556635672020287, 'building': 0.9809385885944656, 'signboard': 0.27702702702702703, 'railing': 0.01910828025477707, 'ashbin': 0.0, 'sidewalk': 0.5430809399477807, 'streetlight': 0.5531914893617021, 'stairway': 0.6986027944111777, 'person': 0.5}, {'tree': 0.8794378698224852, 'building': 0.9819953814239305, 'signboard': 0.3344594594594595, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.4595300261096606, 'streetlight': 0.6212765957446809, 'stairway': 0.7065868263473054, 'person': 0.5416666666666666}, {'tree': 0.8872569737954353, 'building': 0.9836784218560414, 'signboard': 0.30405405405405406, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.3681462140992167, 'streetlight': 0.7021276595744681, 'stairway': 0.720558882235529, 'person': 0.0}, {'tree': 0.900570583262891, 'building': 0.9831304552037262, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.30678851174934724, 'streetlight': 0.7531914893617021, 'stairway': 0.6806387225548902, 'person': 0.0}, {'tree': 0.8877852916314455, 'building': 0.9848917765861678, 'signboard': 0.3108108108108108, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.16318537859007834, 'streetlight': 0.3574468085106383, 'stairway': 0.6427145708582834, 'person': 0.0}, {'tree': 0.849323753169907, 'building': 0.9847352146855063, 'signboard': 0.0, 'railing': 0.0, 'ashbin': 0.0, 'sidewalk': 0.08224543080939947, 'streetlight': 0.0425531914893617, 'stairway': 0.4750499001996008, 'person': 0.0}]
k_runs_per_label = {}
for k_run in accuracies_labels:
    for label, accuracy in k_run.items():
        if label in k_runs_per_label:
            k_runs_per_label[label] += [accuracy]
        else:
            k_runs_per_label[label] = [accuracy]

In [27]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.clf()

list_2D = []
labels = []

for label, accuracies in k_runs_per_label.items():
    list_2D.append(accuracies)
    labels.append(label)

list_2D = list(reversed(list_2D))
labels = list(reversed(labels))

accuracies = np.array(list_2D)

fig, ax = plt.subplots()

fig.tight_layout()
# remove x ticks
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
plt.xlabel('k-Count')
#plt.ylabel('Label')

plt.pcolormesh(accuracies, cmap=plt.get_cmap("RdYlGn"), edgecolors='k', linewidth=1)
ax = plt.gca()
ax.set_aspect('equal')

color_dict = {0: 'normal', 1: 'high', 2: 'very\nhigh'}
for i in range(accuracies.shape[0]):
    for j in range(accuracies.shape[1]):
        plt.text(j+.5, i+.5, str(int(round(accuracies[i, j], 2)*100))+"%", ha="center", va="center", fontsize=8)


plt.yticks(list([x+.5 for x in range(len(labels))]), labels, fontsize=8)
plt.xticks(list([y+.5 for y in range(9)]), [1, 2, 4, 8, 16, 32, 64, 128, 256], fontsize=8)




plt.tight_layout()
plt.savefig("knn_table.svg", bbox_inches='tight')

## Visualizing Different KNN weights

In [107]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.clf()

k=50
# Plot between -10 and 10 with .001 steps.
x_axis = np.arange(1, k+1, 1)
# Mean = 0, SD = 2.
plt.plot(x_axis, [1]*k, label="no weight")
plt.plot(x_axis, np.flip(x_axis/k), label="linear")
plt.plot(x_axis, norm.pdf(x_axis,0,k*0.4)*k, label="normal")
y_axis = [1/(1+d**2) for d in x_axis]
y_axis[0] = 1
plt.plot(x_axis, y_axis, label="1/(1+d²)")
y_axis = [1/(1+d) for d in x_axis]
y_axis[0] = 1
plt.plot(x_axis, y_axis, label="1/(1+d)")
plt.plot(x_axis, [1-d for d in (x_axis/k)**2], label="1-d²")
plt.plot(x_axis, [1-d for d in (x_axis/k)**3], label="1-d³")
plt.ylim(ymin=0)

plt.legend()
plt.xlabel('distance')
plt.ylabel('weight')

#plt.xticks(x_axis, list([x+1 for x in x_axis]))

plt.savefig("weights.png", bbox_inches='tight', dpi=200)

## Backprojection

In [ ]:
labels = {'[4, 200, 3]': 'tree',
         '[180, 120, 120]': 'building',
         '[255, 5, 153]': 'signboard',
         '[255, 61, 6]': 'railing',
         '[173, 0, 255]': 'ashbin',
         '[235, 255, 7]': 'sidewalk',
         '[0, 71, 255]': 'streetlight',
         '[31, 0, 255]': 'stairway',
         '[150, 5, 61]': 'person'}
colors = {'tree': [4, 200, 3],
         'building': [180, 120, 120],
         'signboard': [255, 5, 153],
         'railing': [255, 61, 6],
         'ashbin': [173, 0, 255],
         'sidewalk': [235, 255, 7],
         'streetlight': [0, 71, 255],
         'stairway': [31, 0, 255],
         'person': [150, 5, 61]}


# labels we consider identical
pairs = [([4, 200, 3], [204, 255, 4]), # tree and plant
        ([31, 0, 255], [255, 224, 0]), # stairway and stairs
        ([255, 184, 6], [255, 61, 6])] # fence and railing
def considered_identical(pair):
    return pair in pairs or (pair[1], pair[0]) in pairs

def return_label(x):
    if x==[4,200,3]:
        return 'Tree'
    elif x==[180,120,120]:
        return 'Building'
    elif x==[255,5,153]:
        return 'Signboard'
    elif x==[255,61,6]:
        return 'Railing'
    elif x== [173, 0, 255]:
        return 'Trash Bin'
    elif x==[235, 255, 7]:
        return 'Sidewalk'
    elif x==[0,71,255]:
        return 'Streetlight'
    elif x==[31,0,255]:
        return 'Stairway'
    elif x==[150,5,61]:
        return 'Person'
    elif x== [204, 255, 4]:
        return 'Tree'
    elif x== [255,244,0]:
        return 'Stairway'
    elif x== [255,184,6]:
        return 'Stairway'
    else:
        return 'None'


def obtain_labels(points):
    pred = []
    gt = []

    for point_id in points:
        point = points[point_id]
        pt_col = point.rgb
        gt_col = list(color_dict[point_id])
        pred.append(return_label(pt_col))
        gt.append(return_label(gt_col))
    return pred, gt


def check_against_ground_truth_per_label_images(images):
    mask_colors = [[0, 0, 0], [120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],\
         [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255], [230, 230, 230], [4, 250, 7],\
             [224, 5, 255], [235, 255, 7], [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],\
                 [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3], [0, 102, 200], [61, 230, 250],\
                     [255, 6, 51], [11, 102, 255], [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220], \
                        [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224], [255, 184, 6], [10, 255, 71],\
                             [255, 41, 10], [7, 255, 255], [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],\
                                 [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153], [6, 51, 255], [235, 12, 255], \
                                    [160, 150, 20], [0, 163, 255], [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],\
                                         [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255], [255, 71, 0], [0, 235, 255], \
                                            [0, 173, 255], [31, 0, 255], [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],\
                                                 [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0], [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0], [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255], [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255], [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20], [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255], [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255], [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255], [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0], [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0], [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255], [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112], [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160], [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163], [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0], [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0], [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255], [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204], [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255], [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255], [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194], [102, 255, 0], [92, 0, 255]]
    preds= []
    gt = []
    for image_id in images:
        image = images[image_id]
        xys = image.xys
        point3D_ids = image.point3D_ids
        data = None
        with open(masks_path/Path(image.name.replace(".JPG", ".pkl")), 'rb') as f:
            data = pickle.load(f)[0]
    
        for i, point2D in enumerate(image.xys):
            # get certainties
            pkl_x, pkl_y = get_pkl_coordinates((int(point2D[1]), int(point2D[0])))
            certainties_raw = data[pkl_x][pkl_y]
            certainties_raw[3] = min(certainties_raw)
            mask_color = np.argmax(certainties_raw)
            rgb = mask_colors[mask_color]
            try:
                preds.append(return_label(rgb))
                true_color = color_dict[image.point3D_ids[i]]
                true_color = list(true_color)
                gt.append(return_label(true_color))
            except:
                pass
        return preds, gt

def get_per_class(gt, preds):
    labels = list(set(gt+preds))
    acc = {}
    tot_labels = {}
    corr_labels = {}
    for label in labels:
        total_labels = len([x for x in gt if x==label])
        tot_labels[label] = total_labels
        correct_labels = len([x for x,y in zip(gt, preds) if x==label and y==label])
        corr_labels[label] = correct_labels
        if total_labels!=0:
            acc[label] = correct_labels/total_labels*100
        else:
            acc[label] = None
    return acc, corr_labels, tot_labels